# 🐍 Basilisk: Forensic Video Tracking & Active Defense

This notebook demonstrates the capabilities of Basilisk, including:
1. **Perceptual Hash Extraction** - Fingerprinting videos
2. **Compression Robustness** - Verifying hash stability under attack
3. **Active Defense (Poisoning)** - Forcing a video to match a target hash

In [ ]:
import sys
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

# Add project root to path
sys.path.insert(0, '../')

from core.perceptual_hash import load_video_frames, extract_perceptual_features, compute_perceptual_hash, hamming_distance
from core.adversarial import poison_video

def display_frames(frame_list, title="Video Frames"):
    plt.figure(figsize=(15, 5))
    for i, frame in enumerate(frame_list[:5]):
        plt.subplot(1, 5, i+1)
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

## 1. Perceptual Hash Extraction

We extract a 256-bit hash that represents the visual content of the video, resistant to compression.

In [ ]:
# Create a dummy test video if needed
if not os.path.exists('demo.mp4'):
    !python3 ../experiments/make_short_test_video.py
    !mv short_test.mp4 demo.mp4

print("loading video...")
frames = load_video_frames('demo.mp4', max_frames=30)
display_frames(frames, "Original Video")

print("Extracting hash...")
features = extract_perceptual_features(frames)
original_hash = compute_perceptual_hash(features)
print(f"Original Hash (first 64 bits): {''.join(map(str, original_hash[:64]))}...")

## 2. Active Defense (Adversarial Poisoning)

Now we generate a **TARGET** hash (random signature) and force the video to masquerade as it.

In [ ]:
# 1. Generate Random Target
np.random.seed(123)
target_hash = np.random.randint(0, 2, 256)
print(f"Target Signature (first 64 bits): {''.join(map(str, target_hash[:64]))}...")

# 2. Check initial distance
dist_before = hamming_distance(original_hash, target_hash)
print(f"Distance Before Poisoning: {dist_before}/256 bits ({(dist_before/256)*100:.1f}%)")

# 3. POISON THE VIDEO
print("\n🚀 Launching Poisoning Attack (PGD)...")
poison_video(
    'demo.mp4', 
    target_hash, 
    'protected.mp4', 
    epsilon=0.05, 
    num_iterations=100,
    verbose=True
)
print("✅ Poisoning Complete.")

## 3. Verification

We calculate the hash of the **poisoned** video and check if it matches the target.

In [ ]:
frames_poisoned = load_video_frames('protected.mp4', max_frames=30)
features_poisoned = extract_perceptual_features(frames_poisoned)
final_hash = compute_perceptual_hash(features_poisoned)

dist_after = hamming_distance(final_hash, target_hash)
print(f"\nDistance After Poisoning: {dist_after}/256 bits ({(dist_after/256)*100:.1f}%)")

is_match = dist_after < 30
print(f"Match Confirmed: {'✅ YES' if is_match else '❌ NO'}")

display_frames(frames_poisoned, "Protected Video (Poisoned)")